In [ ]:
import rioxarray
import rioxarray.merge
import xarray
import geopandas
import shapely.geometry
import shapely.ops
import rasterio
import numpy
import matplotlib.pyplot
import pathlib
import pdal
import json

from IPython.core.display import display, HTML
display(HTML("<style>.container { width:100% !important; }</style>"))

# Setup paths

In [ ]:
backgound_path = pathlib.Path(r'C:\Users\pearsonra\Documents\data')
base_path = pathlib.Path(r'C:\Users\pearsonra\Documents\data\Wakanae\Small_test_site')
initial_path = pathlib.Path(r'initial_data')
destination_path = pathlib.Path(r'final_data')

if not (base_path/destination_path).is_dir():
    (base_path/destination_path).mkdir()

boundary_name = pathlib.Path(r'boundary\boundary.shp')
coast_name = pathlib.Path(r'gis\coast\lds-nz-coastlines-and-islands-polygons-topo-150k-SHP.zip!nz-coastlines-and-islands-polygons-topo-150k.shp')
background_dem_name = pathlib.Path(r'DEMs\BackgroundDEM\NZDEM_SoS_15m\NZDEM_SoS_v1-0_16_Wellington_gf.tif')
bathymetry_name_countour = pathlib.Path(r'C:\Users\pearsonra\Documents\data\Bathymetry\Waikanae\lds-depth-contour-polyline-hydro-122k-190k-SHP.zip!depth-contour-polyline-hydro-122k-190k.shp')
bathymetry_name_countour_low_res = pathlib.Path(r'C:\Users\pearsonra\Documents\data\Bathymetry\Waikanae\lds-depth-contour-polyline-hydro-190k-1350k-SHP.zip!depth-contour-polyline-hydro-190k-1350k.shp')
bathymetry_name_points = pathlib.Path(r'background_bathymetry\sounding-points-hydro-122k-190k.shp')
lidar_name = pathlib.Path(r'points.laz')

destination_points_name = pathlib.Path(r'combined_points')
destination_dem_name_stub = pathlib.Path(r'created_dem')

# Set resolution

In [ ]:
resolution = 10
foreshore_buffer = 2
island_name = 'North Island or Te Ika-a-Māui'

# Load in data

In [ ]:
boundary_catchment = geopandas.read_file(base_path/initial_path/boundary_name)
land = geopandas.read_file(backgound_path/coast_name)
bathy_countours = geopandas.read_file(backgound_path/bathymetry_name_countour_low_res)
bathy_points = geopandas.read_file(base_path/initial_path/bathymetry_name_points)
bkgnd_dem = rioxarray.rioxarray.open_rasterio(backgound_path/background_dem_name, masked=True)

### Filter out other islands

In [ ]:
land = land[land['name'] ==island_name].reset_index(drop=True)

# Set to same projection

In [ ]:
crs = 2193
boundary_catchment = boundary_catchment.to_crs(crs)
land = land.to_crs(crs)
bathy_countours = bathy_countours.to_crs(crs)
bathy_points = bathy_points.to_crs(crs)
bkgnd_dem.rio.set_crs(crs);

In [ ]:
f, ax = matplotlib.pyplot.subplots(figsize=(15, 10))

land.plot(ax=ax, color='None', edgecolor='brown', linewidth=5)
boundary_catchment.plot(ax=ax, color='None', edgecolor='green', linewidth=5)
bathy_points.plot(ax=ax, marker='o', color='blue', markersize=5)
bathy_countours.plot(ax=ax, marker='o', color='red', markersize=5)
matplotlib.pyplot.xlim([1.6e6, 2.1e6])
matplotlib.pyplot.ylim([5.4e6, 6e6])
ax.set_title("Uncut data")

# Clip data to in catchment

In [ ]:
catchment_land = geopandas.clip(boundary_catchment, land)
bathy_points = geopandas.clip(bathy_points, boundary_catchment)
bathy_points = bathy_points.reset_index(drop=True)
bathy_countours = geopandas.clip(bathy_countours, boundary_catchment)
bathy_countours = bathy_countours.reset_index(drop=True)
bkgnd_dem = bkgnd_dem.rio.clip(boundary_catchment.geometry)

In [ ]:
f, ax = matplotlib.pyplot.subplots(figsize=(15, 10))

land.plot(ax=ax, color='None', edgecolor='brown', linewidth=5)
boundary_catchment.plot(ax=ax, color='None', edgecolor='green', linewidth=5)
bathy_points.plot(ax=ax, marker='o', color='blue', markersize=5)
bathy_countours.plot(ax=ax, marker='o', color='red', markersize=5)
matplotlib.pyplot.xlim([1.765e6, 1.776e6])
matplotlib.pyplot.ylim([5.469e6, 5.475e6])
ax.set_title("Uncut data")

# Define raster origin and size

In [ ]:
raster_origin = [boundary_catchment.loc[0].geometry.bounds[0], boundary_catchment.loc[0].geometry.bounds[1]]
raster_size = [int((boundary_catchment.loc[0].geometry.bounds[2] - boundary_catchment.loc[0].geometry.bounds[0]) / resolution), int((boundary_catchment.loc[0].geometry.bounds[3] - boundary_catchment.loc[0].geometry.bounds[1]) / resolution)]

# Load in LiDAR and trim to boundary

In [ ]:
lidar_file_name = base_path/initial_path/lidar_name

pdal_pipeline_instructions = [
    {"type":  "readers.las", "filename": str(lidar_file_name)},
    {"type":"filters.reprojection","out_srs":"EPSG:" + str(crs)}, # reproject to NZTM
    {"type":"filters.crop", "polygon":str(boundary_catchment.loc[0].geometry)}, # filter within boundary
    {"type" : "filters.hexbin"} # create a polygon boundary of the LiDAR
]

pdal_pipeline = pdal.Pipeline(json.dumps(pdal_pipeline_instructions))
pdal_pipeline.execute();

lidar_array = pdal_pipeline.arrays[0]

# Define LiDAR extents

In [ ]:
metadata=json.loads(pdal_pipeline.get_metadata())
boundary_lidar=shapely.wkt.loads(metadata['metadata']['filters.hexbin']['boundary'])

Filter areas less than some percentage

In [ ]:
percentage_to_drop = 5
area_to_drop = shapely.geometry.Polygon(boundary_lidar.exterior).area * percentage_to_drop / 100
boundary_lidar = shapely.geometry.Polygon(boundary_lidar.exterior.coords,
            [interior for interior in boundary_lidar.interiors if shapely.geometry.Polygon(interior).area > area_to_drop])
boundary_lidar = geopandas.GeoDataFrame(index=[0], geometry=geopandas.GeoSeries([boundary_lidar], crs=crs), crs=crs)

# Filter DEM inside the LiDAR region

In [ ]:
bkgnd_dem = bkgnd_dem.rio.clip([boundary_lidar.loc[0].geometry], invert=True)
bkgnd_dem_on_land = bkgnd_dem.rio.clip(catchment_land.geometry)

# Create a coast buffer for DEM

In [ ]:
land_and_foreshore = geopandas.GeoDataFrame(index=[0], geometry=catchment_land.buffer(resolution * foreshore_buffer), crs=crs)
land_and_foreshore = geopandas.clip(boundary_catchment, land_and_foreshore)
foreshore = geopandas.overlay(land_and_foreshore, catchment_land, how='difference')
foreshore_and_offshore = geopandas.overlay(boundary_catchment, catchment_land, how='difference')
offshore = geopandas.overlay(boundary_catchment, land_and_foreshore, how='difference')

# DEM coast buffer

Trim the DEM inside the coast buffer region but not where there is LiDAR.

In [ ]:
foreshore_with_lidar = geopandas.clip(boundary_lidar, foreshore)
foreshore_without_lidar = geopandas.overlay(foreshore, foreshore_with_lidar, how="difference")

Trim the background DEM to within the coast buffer where there is no LiDAR

In [ ]:
bkgnd_dem_foreshore = bkgnd_dem.rio.clip(foreshore_without_lidar.geometry)

Set the LiDAR and DEM values in the buffer region to zero (unless already negative)

In [ ]:
bkgnd_dem_foreshore.data[0][bkgnd_dem_foreshore.data[0]>0] = 0

# Combined land and foreshore values
Note - there is the possibility this will miss portions of coast if the background DEM or LiDAR is not defined out to sea. In this case we will need to set these values to zero - could set values in DEM prior to cutting.

In [ ]:
dem_x, dem_y = numpy.meshgrid(bkgnd_dem_on_land.x, bkgnd_dem_on_land.y)
dem_z = bkgnd_dem_on_land.data[0].flatten()
dem_land_x = dem_x.flatten()[~numpy.isnan(dem_z)]
dem_land_y = dem_y.flatten()[~numpy.isnan(dem_z)]
dem_land_z = dem_z[~numpy.isnan(dem_z)]


dem_x, dem_y = numpy.meshgrid(bkgnd_dem_foreshore.x, bkgnd_dem_foreshore.y)
dem_z = bkgnd_dem_foreshore.data[0].flatten()
dem_foreshore_x = dem_x.flatten()[~numpy.isnan(dem_z)]
dem_foreshore_y = dem_y.flatten()[~numpy.isnan(dem_z)]
dem_foreshore_z = dem_z[~numpy.isnan(dem_z)]

dem_points = numpy.zeros_like(lidar_array, shape=[len(dem_land_x) + len(dem_foreshore_x)])
dem_points['X'] = numpy.concatenate([dem_land_x, dem_foreshore_x])
dem_points['Y'] = numpy.concatenate([dem_land_y, dem_foreshore_y])
dem_points['Z'] = numpy.concatenate([dem_land_z, dem_foreshore_z])

combined_dense_points_array = numpy.concatenate([lidar_array, dem_points])

# Create Raster where dense data
Land and coast buffer

In [ ]:
dem_file_name_stub = base_path/destination_path/destination_dem_name_stub

window_size = 0
idw_power = 2
radius =  resolution * numpy.sqrt(2)

pdal_pipeline_instructions = [
    {"type":  "writers.gdal", "resolution": resolution, "gdalopts": "a_srs=EPSG:" + str(crs), "output_type":["idw"], 
     "filename": str(dem_file_name_stub) + "_dense_only_window_" + str(window_size) + "_power_" + str(idw_power) + "_radius_" + str(radius) + ".tiff", 
     "window_size": window_size, "power": idw_power, "radius": radius, 
     "origin_x": raster_origin[0], "origin_y": raster_origin[1], "width": raster_size[0], "height": raster_size[1]}
]

pdal_pipeline = pdal.Pipeline(json.dumps(pdal_pipeline_instructions), [combined_dense_points_array])
pdal_pipeline.execute();


# Create foreshore/sea-ward buffer from dense DEM

In [ ]:
boundary_dense_dem = geopandas.GeoDataFrame(index=[0], geometry=geopandas.GeoSeries(shapely.ops.cascaded_union([land_and_foreshore.loc[0].geometry, boundary_lidar.loc[0].geometry])), crs=crs)
offshore_dense_dem = geopandas.overlay(boundary_catchment, boundary_dense_dem, how='difference')
deflated_dense_data = geopandas.GeoDataFrame(index=[0], geometry=boundary_dense_dem.buffer(resolution * -1 * foreshore_buffer), crs=crs)
offshore_edge_dense_dem = geopandas.overlay(boundary_dense_dem, deflated_dense_data, how='difference')
offshore_edge_dense_dem = geopandas.clip(offshore_edge_dense_dem, foreshore_and_offshore)

# Load in the generated dense DEM

In [ ]:
metadata=json.loads(pdal_pipeline.get_metadata())
dense_dem = rioxarray.rioxarray.open_rasterio(metadata['metadata']['writers.gdal']['filename'][0], masked=True)
dense_dem.rio.set_crs(crs);

dense_dem_foreshore = dense_dem.rio.clip(offshore_edge_dense_dem.geometry)

In [ ]:
f, ax = matplotlib.pyplot.subplots(figsize=(15, 10))
dense_dem_foreshore.plot()

# Cut bathy to outside dense region

In [ ]:
bathy_points = geopandas.clip(bathy_points, offshore_dense_dem)
bathy_points = bathy_points.reset_index(drop=True)

bathy_countours = geopandas.clip(bathy_countours, offshore_dense_dem)
bathy_countours = bathy_countours.reset_index(drop=True)

In [ ]:
f, ax = matplotlib.pyplot.subplots(figsize=(15, 10))

land.plot(ax=ax, color='None', edgecolor='brown', linewidth=5)
boundary_catchment.plot(ax=ax, color='None', edgecolor='green', linewidth=5)
bathy_points.plot(ax=ax, marker='o', color='blue', markersize=5)
bathy_countours.plot(ax=ax, marker='o', color='red', markersize=5)
matplotlib.pyplot.xlim([1.765e6, 1.776e6])
matplotlib.pyplot.ylim([5.469e6, 5.475e6])
ax.set_title("Uncut data")

# Subsample from bathy contours
Overwrite the original data - can't run this line twice

In [ ]:
bathy_countours['points']=bathy_countours.geometry.apply(lambda row : shapely.geometry.MultiPoint([ row.interpolate(i * resolution) for i in range(int(numpy.ceil(row.length/resolution)))]))

# Combine Bathymetry and coastal buffer raster

In [ ]:
lidar_array = pdal_pipeline.arrays[0]

dem_x, dem_y = numpy.meshgrid(dense_dem_foreshore.x, dense_dem_foreshore.y)
dem_z = dense_dem_foreshore.data[0].flatten()
dense_dem_foreshore_x = dem_x.flatten()[~numpy.isnan(dem_z)]
dense_dem_foreshore_y = dem_y.flatten()[~numpy.isnan(dem_z)]
dense_dem_foreshore_z = dem_z[~numpy.isnan(dem_z)]

'''bathymetry_x = bathy_points.apply(lambda x : x['geometry'][0].x,axis=1).to_numpy()
bathymetry_y = bathy_points.apply(lambda x : x['geometry'][0].y,axis=1).to_numpy()
bathymetry_z = bathy_points.apply(lambda x : x['geometry'][0].z,axis=1).to_numpy() * -1 # map depth to elevatation'''

bathy_x = numpy.concatenate(bathy_countours['points'].apply(lambda row : [row[i].x for i in range(len(row))]).to_list())
bathy_y = numpy.concatenate(bathy_countours['points'].apply(lambda row : [row[i].y for i in range(len(row))]).to_list())
bathy_z = numpy.concatenate(bathy_countours.apply(lambda row : (row['valdco'] * numpy.ones(len(row['points']))), axis=1).to_list()) * -1 # map depth to elevatation

offshore_x = numpy.concatenate([dense_dem_foreshore_x, bathy_x])
offshore_y = numpy.concatenate([dense_dem_foreshore_y, bathy_y])
offshore_z = numpy.concatenate([dense_dem_foreshore_z, bathy_z])

# Create offshore raster - Plot results
Using `scipy.interpolate` and `scipy.interpolate.rbf`

In [ ]:
import scipy.interpolate

In [ ]:
offshore_dem_linear=scipy.interpolate.griddata(points=(offshore_x, offshore_y), values=offshore_z, xi=(dem_x, dem_y), method='linear')
offshore_dem_nn=scipy.interpolate.griddata(points=(offshore_x, offshore_y), values=offshore_z, xi=(dem_x, dem_y), method='nearest')
offshore_dem_cubic=scipy.interpolate.griddata(points=(offshore_x, offshore_y), values=offshore_z, xi=(dem_x, dem_y), method='cubic')

In [ ]:
vmin = -5
vmax = 1
f, ax = matplotlib.pyplot.subplots(figsize=(15, 10))
ax = matplotlib.pyplot.subplot(131)
matplotlib.pyplot.imshow(offshore_dem_linear, vmin=vmin, vmax=vmax)
ax.set_title('Linear')

ax = matplotlib.pyplot.subplot(132)
matplotlib.pyplot.imshow(offshore_dem_nn, vmin=vmin, vmax=vmax)
ax.set_title('Nearest neighbour')

ax = matplotlib.pyplot.subplot(133)
matplotlib.pyplot.imshow(offshore_dem_cubic, vmin=vmin, vmax=vmax)
ax.set_title('Cubic')

In [ ]:
offshore_rbf = scipy.interpolate.Rbf(offshore_x, offshore_y, offshore_z, function='multiquadric')
offshore_rbf_dem_multiquadric = offshore_rbf(dem_x.flatten(), dem_y.flatten())
offshore_rbf = scipy.interpolate.Rbf(offshore_x, offshore_y, offshore_z, function='inverse')
offshore_rbf_dem_inverse = offshore_rbf(dem_x.flatten(), dem_y.flatten())
offshore_rbf = scipy.interpolate.Rbf(offshore_x, offshore_y, offshore_z, function='gaussian')
offshore_rbf_dem_gaussian = offshore_rbf(dem_x.flatten(), dem_y.flatten())
offshore_rbf = scipy.interpolate.Rbf(offshore_x, offshore_y, offshore_z, function='linear')
offshore_rbf_dem_linear = offshore_rbf(dem_x.flatten(), dem_y.flatten())
offshore_rbf = scipy.interpolate.Rbf(offshore_x, offshore_y, offshore_z, function='cubic')
offshore_rbf_dem_cubic = offshore_rbf(dem_x.flatten(), dem_y.flatten())
offshore_rbf = scipy.interpolate.Rbf(offshore_x, offshore_y, offshore_z, function='quintic')
offshore_rbf_dem_quintic = offshore_rbf(dem_x.flatten(), dem_y.flatten())
offshore_rbf = scipy.interpolate.Rbf(offshore_x, offshore_y, offshore_z, function='thin_plate')
offshore_rbf_dem_thin_plate = offshore_rbf(dem_x.flatten(), dem_y.flatten())

In [ ]:
f, ax = matplotlib.pyplot.subplots(figsize=(30, 15))
ax=matplotlib.pyplot.subplot(241)
matplotlib.pyplot.imshow(offshore_rbf_dem_multiquadric.reshape(dem_x.shape), vmin=vmin, vmax=vmax)
ax.set_title('multiquadric')

ax=matplotlib.pyplot.subplot(242)
matplotlib.pyplot.imshow(offshore_rbf_dem_inverse.reshape(dem_x.shape), vmin=vmin, vmax=vmax)
ax.set_title('inverse')

ax=matplotlib.pyplot.subplot(243)
matplotlib.pyplot.imshow(offshore_rbf_dem_gaussian.reshape(dem_x.shape), vmin=vmin, vmax=vmax)
ax.set_title('gaussian')

ax=matplotlib.pyplot.subplot(244)
matplotlib.pyplot.imshow(offshore_rbf_dem_linear.reshape(dem_x.shape), vmin=vmin, vmax=vmax)
ax.set_title('linear')

ax=matplotlib.pyplot.subplot(245)
matplotlib.pyplot.imshow(offshore_rbf_dem_cubic.reshape(dem_x.shape), vmin=vmin, vmax=vmax)
ax.set_title('cubic')

ax=matplotlib.pyplot.subplot(246)
matplotlib.pyplot.imshow(offshore_rbf_dem_quintic.reshape(dem_x.shape), vmin=vmin, vmax=vmax)
ax.set_title('quintic')

ax=matplotlib.pyplot.subplot(247)
matplotlib.pyplot.imshow(offshore_rbf_dem_thin_plate.reshape(dem_x.shape), vmin=vmin, vmax=vmax, extent=(dem_x.min(), dem_x.max(), dem_y.min(), dem_y.max()))
#matplotlib.pyplot.scatter(offshore_x, offshore_y, c=offshore_z, vmin=vmin, vmax=vmax)#, 'k.', ms=1)
ax.set_title('thin_plate')

# Create offshore raster - Linear RBF
Using `scipy.interpolate.rbf`
Only calculate where offshore

1. Get offshore grid locations to save on RBF interpolant calculations - do some geometric operations to produce an offshore boundary

In [ ]:
offshore_dem=dense_dem.copy()
offshore_dem.rio.set_crs(dense_dem.rio.crs)
offshore_dem.data[0]=0
offshore_dem = offshore_dem.rio.clip(offshore_dense_dem.geometry);

2. Create the RBF function from the offshore data

In [ ]:
offshore_rbf = scipy.interpolate.Rbf(offshore_x, offshore_y, offshore_z, function='linear')

3. Evaluate the RBF function where offshore

In [ ]:
dem_x, dem_y = numpy.meshgrid(offshore_dem.x, offshore_dem.y)
dem_z = offshore_dem.data[0].flatten()
dem_offshore_x = dem_x.flatten()[~numpy.isnan(dem_z)]
dem_offshore_y = dem_y.flatten()[~numpy.isnan(dem_z)]
dem_z[~numpy.isnan(dem_z)] = offshore_rbf(dem_offshore_x, dem_offshore_y)
offshore_dem.data[0] = dem_z.reshape(dem_x.shape)

4. Plot results to sanity check

In [ ]:
offshore_dem.plot()

In [ ]:
dense_dem.plot()

# Combine and fill DEMs

In [ ]:
combined_dem = rioxarray.merge.merge_arrays([dense_dem, offshore_dem], method= "last") # important for this to be last as otherwise values that
combined_dem_filled = combined_dem.rio.interpolate_na()

In [ ]:
combined_dem_filled.plot(vmin=-5, vmax=5)

# Save results

In [ ]:
offshore_dem.to_netcdf(str(dem_file_name_stub) + "_offshore_linear_rbf_dem.nc")
combined_dem.to_netcdf(str(dem_file_name_stub) + "_combined_linear_rbf_dem.nc")
combined_dem_filled.to_netcdf(str(dem_file_name_stub) + "_combined_linear_rbf_dem_filled.nc")